<a href="https://colab.research.google.com/github/Anirudhk07/KCC_Queries_Analysis/blob/master/KCC_Queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
print(tf.__version__)

2.2.0


In [0]:
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_portion = .8

In [3]:


stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
print(len(stopwords))
# Expected Output
# 153

153


In [5]:
from google.colab import files
uploaded = files.upload()

Saving kcc_maharashtra.csv to kcc_maharashtra.csv


In [7]:
import pandas as pd
import io

df = pd.read_csv(io.StringIO(uploaded['kcc_maharashtra.csv'].decode('utf-8')))
print(df.head())

   ï»¿Season       Sector  ... BlockName                CreatedOn
0        NaN  AGRICULTURE  ...    MUMBAI   2019-04-01T10:39:58.58
1        NaN  AGRICULTURE  ...    MUMBAI  2019-04-01T12:06:16.667
2        NaN  AGRICULTURE  ...    MUMBAI   2019-04-01T12:26:50.62
3        NaN  AGRICULTURE  ...    MUMBAI   2019-04-01T15:01:53.64
4        NaN  AGRICULTURE  ...    MUMBAI  2019-04-02T13:33:43.467

[5 rows x 11 columns]


In [17]:
sentences = []
label = []
labels = []
with open("kcc_maharashtra.csv",'r') as csvfile:
    reader = csv.reader(csvfile, delimiter = ',')
    next(reader)
    for row in reader:
        label.append(row[4])
        sentence = row[5]
        for word in stopwords:
            token = " " + word + " "
            sentence = sentence.replace(token, " ")
            sentence = sentence.replace("  ", " ")
        sentences.append((sentence))


def remove(string): 
    s =""
    for i in string:
      if i not in  ('(', '*', '.', ',', '[') :
         s+=i
      else :
        return "".join(s.split())
    return "".join(s.split())
  
for word in label:
    x = remove(word)
    labels.append(x)

print(labels[31])


AnimalProduction


In [18]:
train_size = int(len(sentences) * training_portion)

train_sentences = sentences[:train_size]
train_labels = labels[:train_size]

validation_sentences = sentences[train_size:]
validation_labels = labels[train_size:]

print(train_size)
print(len(train_sentences))
print(len(train_labels))
print(len(validation_sentences))
print(len(validation_labels))

# Expected output (if training_portion=.8)
# 1780
# 1780
# 1780
# 445
# 445

41063
41063
41063
10266
10266


In [33]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

print(len(train_sequences[0]))
print(len(train_padded[0]))

print(len(train_sequences[1]))
print(len(train_padded[1]))

print(len(train_sequences[10]))
print(len(train_padded[10]))
print(train_sequences[0])
# Expected Ouput
# 10
# 120
# 5
# 120
# 6
# 120

10
120
5
120
6
120
[3, 2, 1, 251, 1, 1, 22, 20, 114, 228]


In [20]:
validation_sequences = tokenizer.texts_to_sequences(validation_sentences)
validation_padded = pad_sequences(validation_sequences, padding=padding_type, maxlen=max_length)

print(len(validation_sequences))
print(validation_padded.shape)

# Expected output
# 445
# (445, 120)

10266
(10266, 120)


In [21]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)
print(len(labels))



training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))

validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))
print(type(training_label_seq))
training_label_seq.reshape(41063,1)
validation_label_seq.reshape(10266,1)

print(training_label_seq[0])
print(training_label_seq[1])
print(training_label_seq[2])
print(training_label_seq.shape)

print(validation_label_seq[0])
print(validation_label_seq[1])
print(validation_label_seq[2])
print(validation_label_seq.shape)
print(validation_labels[100])

51329
<class 'numpy.ndarray'>
[3]
[3]
[3]
(41063,)
[1]
[9]
[1]
(10266,)
Weather


In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           16000     
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 24)                408       
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 150       
Total params: 16,558
Trainable params: 16,558
Non-trainable params: 0
_________________________________________________________________


In [23]:
print(labels[0])
num_epochs = 30

model.fit(train_padded, training_label_seq, epochs=num_epochs, validation_data=(validation_padded, validation_label_seq))

GovernmentSchemes


In [31]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
print(reverse_word_index)
def decode_sentence(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])


{1: '<OOV>', 2: 'asked', 3: 'farmer', 4: 'about', 5: 'of', 6: 'weather', 7: 'asking', 8: 'forecast', 9: 'market', 10: 'attack', 11: 'query', 12: 'on', 13: 'in', 14: 'for', 15: 'rate', 16: 'block', 17: 'fertilizer', 18: 'onion', 19: 'crop', 20: 'number', 21: 'scheme', 22: 'contact', 23: 'varieties', 24: 'government', 25: 'wheat', 26: 'kisan', 27: 'gram', 28: 'spray', 29: 'cotton', 30: 'nidhi', 31: 'control', 32: 'dose', 33: 'management', 34: 'caterpillar', 35: 'sugarcane', 36: 'insurance', 37: 'weed', 38: 'tehsil', 39: 'growth', 40: 'information', 41: 'district', 42: 'maize', 43: 'borer', 44: 'yojana', 45: 'pradhanmantri', 46: 'sanman', 47: 'blight', 48: 'sucking', 49: 'pest', 50: 'leaf', 51: 'tahsil', 52: 'samman', 53: 'mandi', 54: 'brinjal', 55: 'detail', 56: 'pm', 57: 'sorghum', 58: 'turmeric', 59: 'price', 60: 'iffco', 61: 'eating', 62: 'bengal', 63: 'tokio', 64: 'wilt', 65: 'yojna', 66: 'application', 67: 'mantri', 68: 'pradhan', 69: 'pune', 70: 'no', 71: 'fungal', 72: 'parbhani', 

In [25]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

# Expected output
# (1000, 16)

(1000, 16)


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')